In [ ]:
#STILL WIP

# # Read from:
# Dune - OP Deployed by deployer address type 
# Defillama/Subgraphs - TVL Flows by Program
# Notion - OP Budget by Program

# Join these datasets together on program & associate anything else to the generalized programs

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
tvl = pd.read_csv('csv_outputs/op_summer_latest_stats.csv')
distrib_df = pd.read_csv('csv_outputs/dune_op_distribution_type.csv')
program_df = pd.read_csv('inputs/OP Summer Programs [Public].csv')

In [ ]:
#Filter TVL DF
tvl = tvl[tvl['include_in_summary'] == 1]
tvl['join_key'] = tvl['top_level_name'].str.replace('*','') #tvl['app_name'] + ' - ' + tvl['top_level_name'].str.replace('*','')
# display(tvl)

In [ ]:
#Set up Distributions for Mapping
distrib_df['program_map'] = np.where(
                                distrib_df['to_type'].isin(tvl['top_level_name'])
                                ,distrib_df['to_type']
                                ,''
                                )

sum_distrib_df = distrib_df[['from_name','op_deployed','to_label','to_type','program_map']].groupby(['from_name','program_map']).sum()
sum_distrib_df.reset_index(inplace=True)
#Joins should maybe just be the program map OR from name, since Velo operated bribes for a while
sum_distrib_df['join_key'] = np.where(sum_distrib_df['program_map'] == '',sum_distrib_df['from_name'],sum_distrib_df['program_map'])
                            # sum_distrib_df['from_name'] + ' - ' \
                            #     + np.where(sum_distrib_df['program_map'] == '',sum_distrib_df['from_name'],sum_distrib_df['program_map'])

# display(sum_distrib_df[sum_distrib_df['join_key'].str.contains('elodr')])

In [ ]:
#lowercase joinkeys
tvl['join_key'] = tvl['join_key'].str.lower()
sum_distrib_df['join_key'] = sum_distrib_df['join_key'].str.lower()
#
df = sum_distrib_df.merge(tvl, on='join_key',how='outer')
display(df[df['join_key'].str.contains('velodr')])

In [ ]:
# Overrides as needed
def replace_program_names(df, overrides):
    for program, program_override in overrides.items():
        df.loc[df['join_key'] == program, 'from_name'] = program_override
    return df

#Overrides if needed
overrides = {
    'old name'  :   'new name',
}

# Replace program names with overrides
df = replace_program_names(df, overrides)

In [ ]:
#Create the aggregate app name field
df['agg_app_name'] = df['app_name'].combine_first(df['from_name'])
df = df.fillna(0) #Fill NA with 0
# display(df)
display(df[df['agg_app_name'].str.contains('elodr')])

In [ ]:
# Now union back again

select_cols = ['agg_app_name','top_level_name', 'program_name',\
               'num_op_override', 'period','op_source', 'start_date','end_date'
               ,'op_deployed', 
               'cumul_net_dollar_flow_at_program_end',
                'cumul_net_dollar_flow',
                'cumul_last_price_net_dollar_flow_at_program_end',
                'cumul_last_price_net_dollar_flow'
                ]
group_cols = select_cols[:8] #group by 1 to 8
print(group_cols)

# display(df[select_cols])

sum_distrib_df = df[select_cols].groupby(group_cols).sum()
sum_distrib_df.reset_index(inplace=True)

In [ ]:
#Get the rank by start_date of each program

# replace 0s with '9999-12-31'
sum_distrib_df['start_date'] = np.where(sum_distrib_df['start_date'] == 0, '9999-12-31', sum_distrib_df['start_date'])
display(sum_distrib_df)
# create a new column 'program_rank' based on the 'start_date' column
sum_distrib_df['program_rank'] = sum_distrib_df.groupby('agg_app_name')['start_date'].rank(ascending=True)
# replace 'today' with 0s
sum_distrib_df['start_date'] = np.where(sum_distrib_df['start_date'] == '9999-12-31', 0, sum_distrib_df['start_date'])

sum_distrib_df = sum_distrib_df.sort_values(by=['agg_app_name','program_rank'],ascending=[True,True])
# subtract all overridden values fromthe amount I have deployed

# create a new column 'cumulative_num_op_override' that contains the cumulative sum of 'num_op_override' for each agg_app_name group
sum_distrib_df['cumulative_num_op_override'] = sum_distrib_df.groupby('agg_app_name')['num_op_override'].cumsum()
# create a new column 'op_deployed_net_override' that subtracts 'cumulative_num_op_override' from 'op_deployed'
sum_distrib_df['op_deployed_net_override'] = sum_distrib_df['op_deployed'] - sum_distrib_df.groupby('agg_app_name')['cumulative_num_op_override'].shift(1).fillna(0)
# drop the 'cumulative_num_op_override' column
# sum_distrib_df.drop('cumulative_num_op_override', axis=1, inplace=True)

In [ ]:
# Override # OP Deployed 
sum_distrib_df['op_deployed'] = sum_distrib_df.apply(lambda x: x['num_op_override'] if x['num_op_override'] > 0 else x['op_deployed'], axis=1)

# replace 0s in 'num_op_override' with the corresponding value in 'op_deployed_net_override'
sum_distrib_df['og_op_deployed'] = sum_distrib_df['op_deployed']
sum_distrib_df['op_deployed'] = np.where(sum_distrib_df['num_op_override'] == 0, sum_distrib_df['op_deployed_net_override'], sum_distrib_df['num_op_override'])

#Select all except the last 4 rows
sum_distrib_df = sum_distrib_df.iloc[:, :-4]


In [ ]:
# latest tvl metrics
sum_distrib_df['net_flows_retention'] = np.where(sum_distrib_df['period']=='Post-Program'\
            ,sum_distrib_df['cumul_last_price_net_dollar_flow'] / sum_distrib_df['cumul_last_price_net_dollar_flow_at_program_end']\
            ,np.nan\
)

In [ ]:
display(sum_distrib_df)
sum_distrib_df['last_updated'] = pd.to_datetime(datetime.datetime.now())
sum_distrib_df.to_csv('csv_outputs/incentives_stats_summary.csv')
# display(sum_distrib_df[sum_distrib_df['agg_app_name'].str.contains('rrakis')])